This notebook querries the gaia sources around the known GEMS and computes the 3D separation from the target. I found that about 1 in 4 among the known GEMS have a nearby companion. 

TODO: 
* check which GEMS have companions not reported in literature (discovery paper)
* check if these are comoving companions
* check if theory supports that giant planets are more likely to form around binary star disk
  - [Most HJ are lonely](https://ui.adsabs.harvard.edu/abs/2024ESS.....561101S/abstract)
  - [HJ are not as lonely: 72% ± 16% of hot Jupiters are part of multi-planet and/or multi-star systems](https://ui.adsabs.harvard.edu/abs/2015ApJ...800..138N/abstract)
  - [at least 12% ± 6% of hot Jupiters have a nearby planetary companion detected via TTV in Kepler light curves](https://ui.adsabs.harvard.edu/abs/2023AJ....165..171W/abstract)
  - 

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from toi5671 import Star

/opt/miniconda3/envs/toi5671/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import pandas as pd
from pathlib import Path

redownload = False

fp = '../data/nexsci_data.csv'
if redownload or not Path(fp).exists():
    #pstable combines data from the Confirmed Planets and Extended Planet Parameters tables
    tab = NasaExoplanetArchive.query_criteria(table="pscomppars", 
    #                                          where="discoverymethod like 'Transit'"
    )
    df = tab.to_pandas()                                        
    df.to_csv(fp, index=True)
    
else:
    df = pd.read_csv(fp, index_col=0)
    df.index.name = 'index'
df.head()

/var/folders/4s/07r41mpn76n_0fykwqyjgc840000gn/T/ipykernel_98022/1112017024.py:17: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp, index_col=0)


,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,...,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20,sky_coord.ra,sky_coord.dec
index,,,,,,,,,,,,,,,,,,,,,
0,OGLE-2016-BLG-1227L b,b,OGLE-2016-BLG-1227L,NaN,NaN,NaN,2020-03,2020,Microlensing,Ground,...,NaN,NaN,NaN,NaN,-0.063824,-0.828921,-0.555712,-768415656,265.597125,-33.759778
1,Kepler-276 c,c,Kepler-276,NaN,NaN,TIC 138213510,2014-02,2013,Transit,Space,...,7.99553,0.051581,-0.051581,7.9955300&plusmn;0.0515805,0.310575,-0.711954,0.629813,1302657699,293.568197,39.036312
2,Kepler-829 b,b,Kepler-829,NaN,NaN,TIC 123451768,2016-05,2016,Transit,Space,...,7.93777,0.040833,-0.040833,7.9377700&plusmn;0.0408326,0.157566,-0.720680,0.675124,-792260935,282.332831,42.463813
3,K2-283 b,b,K2-283,NaN,NaN,TIC 266017624,2018-12,2018,Transit,Space,...,-10.97340,0.053624,-0.053624,-10.9734000&plusmn;0.0536244,0.959702,0.224997,0.168368,609813555,13.194368,9.692918
4,Kepler-477 b,b,Kepler-477,NaN,NaN,TIC 158633329,2016-05,2016,Transit,Space,...,-26.00000,NaN,NaN,-26.0,0.229185,-0.702544,0.673726,-1968935374,288.067445,42.355305


In [13]:
#HJ
idx1 = (df.st_teff>=4_000) & (df.st_teff<=10_000) & (df.pl_radj > 0.4) & (df.discoverymethod=="Transit") & (df.pl_ratdor < 10) 
hj = df[idx1]
hj.shape

(413, 375)

In [25]:
from tqdm import tqdm
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
import astropy.units as u

gaia = {}
cols = ["ra","dec","parallax","pmra","pmdec","radial_velocity","phot_g_mean_mag","bp_rp","distance"]
for name in tqdm(hj.hostname):
    print(name)
    try:
        star = Star(name)
        gaia_sources = star.get_gaia_sources(rad_arcsec=60)
        # print(gaia_sources[cols])
        
        s = gaia_sources[cols].dropna(subset=["parallax"])
        s = s[s["parallax"]>0]
        coords = SkyCoord(ra=s.ra.values*u.deg, 
                         dec=s.dec.values*u.deg,
                         pm_ra_cosdec=s.pmra.values*u.mas/u.yr,
                         pm_dec=s.pmdec.values*u.mas/u.yr,
                         radial_velocity=s.radial_velocity.values*u.km/u.second,
                         distance=Distance(parallax=s.parallax.values*u.mas)
                        )
        s['distance_3d'] = coords.separation_3d(coords[0]).value
        gaia[name] = s
        # break
    except Exception as e:
        print(e)

  0%|                                                                                                                                                                                       | 0/413 [00:00<?, ?it/s]

WASP-49
Mstar=(0.99,0.12) Msun
Rstar=(1.02,0.05) Rsun
Rhostar=(0.93,0.18) rhosun
teff=(5597,121) K
logg=(4.42,0.08) cgs
feh=(0.00,0.10) dex
Querying Gaia sources 60" around WASP-49: (91.0897, -16.9654) deg.


  0%|▍                                                                                                                                                                              | 1/413 [00:07<51:43,  7.53s/it]

HAT-P-23
Mstar=(1.08,0.14) Msun
Rstar=(1.15,0.06) Rsun
Rhostar=(0.71,0.14) rhosun
teff=(5918,137) K
logg=(4.35,0.08) cgs
feh=(0.00,0.10) dex
Querying Gaia sources 60" around HAT-P-23: (306.1239, 16.7621) deg.


  0%|▊                                                                                                                                                                              | 2/413 [00:16<55:53,  8.16s/it]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 411/413 [50:08<00:13,  7.00s/it]

float() argument must be a string or a real number, not 'NoneType'
Check exofop: https://exofop.ipac.caltech.edu/tess/target.php?id=HAT-P-63&json
NGTS-25


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 412/413 [50:15<00:06,  6.99s/it]

float() argument must be a string or a real number, not 'NoneType'
Check exofop: https://exofop.ipac.caltech.edu/tess/target.php?id=NGTS-25&json
CoRoT-3
Mstar=(1.36,0.22) Msun
Rstar=(1.53,0.09) Rsun
Rhostar=(0.38,0.09) rhosun
teff=(6558,104) K
logg=(4.20,0.09) cgs
feh=(0.00,0.10) dex
Querying Gaia sources 60" around CoRoT-3: (292.0553, 0.1218) deg.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 413/413 [50:25<00:00,  7.33s/it]


In [120]:
data = fk.load("../data/hot_jupiter_companions.h5")
has_nearby_companion = []
for name in data:
    # print(name)
    # print(data[name].sort_values(by="distance_3d").head(3))
    if len(data[name])>1 and data[name].sort_values(by="distance_3d").iloc[1].distance_3d<10:
        has_nearby_companion.append(name)

In [124]:
sorted([data[n].sort_values(by='distance_3d').iloc[1].distance_3d*u.pc.to(u.au) if len(data[n])>1 else 999e6 for n in data])[:10]

[12092.950093862777,
 25759.41067583616,
 30852.422114511042,
 33911.672282315274,
 52881.917963136446,
 82994.22260052364,
 87145.11110741264,
 121006.98463499038,
 124114.19174586436,
 132073.9778706472]

In [ ]:
for name in has_nearby_companion:
    d = data[name]
    ax = d.plot.scatter(x='pmra', y='pmdec', figsize=(5,5))
    d.iloc[:2].plot.scatter(x='pmra', y='pmdec', c='r', ax=ax)
    ax.set_title(name)

In [82]:
len(has_nearby_companion)/len(hj)

0.014527845036319613

## GEMS

In [60]:
#GEMS
idx2 = (df.st_teff<=4000) & (df.pl_radj > 0.5) & (df.discoverymethod=="Transit") & (df.pl_ratdor > 10) 
gems = df[idx2]
gems.shape

(23, 375)

In [ ]:
from tqdm import tqdm
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
import astropy.units as u

gaia = {}
cols = ["ra","dec","parallax","pmra","pmdec","radial_velocity","phot_g_mean_mag","bp_rp","distance"]
for name in tqdm(gems.hostname):
    print(name)
    try:
        star = Star(name)
        gaia_sources = star.get_gaia_sources(rad_arcsec=60)
        # print(gaia_sources[cols])
        
        s = gaia_sources[cols].dropna(subset=["parallax"])
        s = s[s["parallax"]>0]
        coords = SkyCoord(ra=s.ra.values*u.deg, 
                         dec=s.dec.values*u.deg,
                         pm_ra_cosdec=s.pmra.values*u.mas/u.yr,
                         pm_dec=s.pmdec.values*u.mas/u.yr,
                         radial_velocity=s.radial_velocity.values*u.km/u.second,
                         distance=Distance(parallax=s.parallax.values*u.mas)
                        )
        s['distance_3d'] = coords.separation_3d(coords[0]).value
        gaia[name] = s
        # break
    except Exception as e:
        print(e)

In [37]:
ls -lh ../data/gems_companions.h5

-rw-r--r--@ 1 jpdl  staff   125K Apr 22 19:22 ../data/gems_companions.h5


In [125]:
data = fk.load("../data/gems_companions.h5")
has_nearby_companion = []
for name in data:
    # print(name)
    # print(data[name].sort_values(by="distance_3d").head(3))
    if len(data[name])>1 and data[name].sort_values(by="distance_3d").iloc[1].distance_3d<100:
        has_nearby_companion.append(name)

In [126]:
sorted([data[n].sort_values(by='distance_3d').iloc[1].distance_3d*u.pc.to(u.au) if len(data[n])>1 else 999e6 for n in data])[:10]

[262192.6862231671,
 307542.4536065932,
 1943401.9466828804,
 9846753.538170641,
 14705230.24277404,
 19517331.6627828,
 26326031.886921186,
 32131605.992111802,
 38704233.12019201,
 52021012.49292728]

In [134]:
has_nearby_companion

['HATS-74 A', 'Kepler-1624', 'TOI-3235', 'TOI-3714', 'TOI-3757', 'TOI-3984 A']

In [40]:
len(has_nearby_companion)/len(gems)

0.13043478260869565